---
comments: true
layout: post
title: Setting up Cockpit
description: Setting up Cockpit
---

# Install Cockpit

In [ ]:
sudo apt update
sudo apt install cockpit

Start the Cockpit service

In [ ]:
sudo systemctl start cockpit

# Accessing

Cockpit runs on port 9090. This port needs to be allowed on the EC2 instance. Click on "Security Groups" in the left sidebar: 

![]({{ site.baseurl }}/images/ip.jpg)

Next, search for your security group. Click the check box next to the group name, scroll down, and click on "Inbound rules". Click on "Edit inbound rules", and add a rule that allows "Port range" of 9090 and "Source type" of Anywhere-IPv4.
<br>

Access Cockpit with the link: `<EC2 IP address>:9090`. There are a variety of ways to find the IP address of the instance. One way is to locate the IP address under "Public IPv4 address" on the "Instances" page:

![]({{ site.baseurl }}/images/ip.jpg)

In this case, the IP address of the instance is 3.145.69.15. Accessing http://3.145.69.15:9090 shows the Cockpit login screen:

![]({{ site.baseurl }}/images/cockpitHomepage.jpg)

<br>
Login with the same username and password used to login to the EC2 instance.

# Configure subdomain

1. Go to AWS Route 53 -> Hosted zone -> nighthawkcodingsociety.com

2. Click on "Create record"

3. Configure a subdomain, keep the "Record type" as "A", and add the IP address of the instance in the "Value" section.

<br>

Now you can access Cockpit at `<subdomain>:9090`.

# Change Cockpit port number

Create the file /etc/systemd/system/cockpit.socket.d/listen.conf

Check if the directory /etc/systemd/system/cockpit.socket.d/ exists. If not, type:

```bash
sudo mkdir /etc/systemd/system/cockpit.socket.d
```

Then, create and edit the `listen.conf` file. 

```bash
sudo nano /etc/systemd/system/cockpit.socket.d/listen.conf
```

Add the following lines into the file:

```
[Socket]
ListenStream=
ListenStream=<your port here>
```

You can change the port to 443 (if using HTTPS) or 80 (if using HTTP). This way, you can access Cockpit through a URL that doesn't need a port (ex: http://cockpit.nighthawkcodingsociety.com/ )

Afterwards, run:

```bash
sudo systemctl daemon-reload
sudo systemctl restart cockpit.socket
```



# To be continued

Things to research/add:

* Make the domain use HTTPS
* Cockpit features
